In [11]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Artist selection widget
artist_toggle_options = ['At most k songs', 'At least k songs', 'Excluding this artist', 'Excluding this song', '--']
artist_toggle = widgets.Dropdown(
    options=artist_toggle_options,
    description='Preferences:',
    disabled=False,
    value='--'
)

# Number of songs (at most)
at_most_song_count = widgets.IntSlider(
    value=1,
    min=0,
    max=100,  # Adjust maximum as needed
    step=1,
    description='At most k songs:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='25%'),
    style={'description_width': 'initial'}
)

# Number of songs (at least)
at_least_song_count = widgets.IntSlider(
    value=1,
    min=0,
    max=100,  # Adjust maximum as needed
    step=1,
    description='At least k songs:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='25%'),
    style={'description_width': 'initial'}
)

artist_name = widgets.Text(description='Artist name:')
song_name = widgets.Text(description='Song name:')

# artist_and_count = widgets.HBox([artist_toggle])
# artist_and_count.layout.align_items = 'center'

# intialize the output - second widget
out = Output()

def changed(change):
    '''
    Monitor change in the first widget
    '''
    global out

    if artist_toggle.value == 'At most k songs':
        out.append_display_data(at_most_song_count)
#         display(out)
    elif artist_toggle.value == 'At least k songs':
        out.append_display_data(at_least_song_count)
#         display(out)
    elif artist_toggle.value == 'Excluding this artist':
        out.append_display_data(artist_name)
#         display(out)
    elif artist_toggle.value == 'Excluding this song':
        out.append_display_data(song_name)
#         display(out)
    else:
        out.clear_output()
        out = Output()
    
    display(out)
    out.clear_output()   

                
# monitor the friends widget for changes
artist_toggle.observe(changed, 'value')

playlist_length_toggle = widgets.ToggleButtons(
    description='Playlist length:',
    options= ['Minutes', 'Number of Songs'])

# # Playlist length in minutes
# playlist_length_mins = widgets.IntRangeSlider(
#     value=[0, 120],
#     min=0,
#     max=120,  # 2 hours in minutes
#     step=10,  # step in minutes
#     description='Playlist Length (min):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # Playlist length in number of songs
# playlist_length_num_songs = widgets.IntSlider(
#     value=10,
#     min=2,
#     max=100,
#     step=1,
#     description='Playlist Length (number of songs):',
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='d',
#     style={'description_width': 'initial'},
# )

# # intialize the output - second widget
# out = Output()

# def changed(change):
#     '''
#     Monitor change in the first widget
#     '''
#     global out
#     if playlist_length_toggle.value == 'Minutes':  
#         out.append_display_data(playlist_length_mins)
#         # display(out)
#     elif playlist_length_toggle.value == 'Number of Songs':
#         out.append_display_data(playlist_length_num_songs)
#         # display(out)
#     else:
#         out.clear_output()
#         out = Output()
    
#     display(out)
#     out.clear_output()    
                
# monitor the friends widget for changes
playlist_length_toggle.observe(changed, 'value')


# BPM range widget
bpm_range = widgets.IntRangeSlider(
    value=[60, 180],  # Default BPM range
    min=0,
    max=300,
    step=1,
    description='BPM Range:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Loudness widget
loudness_range = widgets.FloatRangeSlider(
    value=[-60.0, 0.0],
    min=-60.0,
    max=0.0,
    step=0.1,
    description='Loudness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Liveness widget
liveness_range = widgets.FloatRangeSlider(
    value=[0.0, 1.0],
    min=0.0,
    max=1.0,
    step=0.01,
    description='Liveness:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

# Mode widget
mode_options = ['Major', 'Minor']
mode_selection = widgets.Dropdown(
    options=mode_options,
    description='Mode:',
    disabled=False,
)

# Display widgets
display(bpm_range, loudness_range, liveness_range, mode_selection, artist_toggle)

IntRangeSlider(value=(60, 180), continuous_update=False, description='BPM Range:', max=300)

FloatRangeSlider(value=(-60.0, 0.0), continuous_update=False, description='Loudness:', max=0.0, min=-60.0, rea…

FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='Liveness:', max=1.0, step=0.01)

Dropdown(description='Mode:', options=('Major', 'Minor'), value='Major')

Dropdown(description='Preferences:', index=4, options=('At most k songs', 'At least k songs', 'Excluding this …

In [3]:
import pandas as pd
import numpy as np

In [4]:
from util import get_model_songs

df = get_model_songs("stay (with justin bieber)", 5000)

2023-12-05 22:38:51.820698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 22:38:51.963297: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 22:38:51.967776: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-05 22:38:51.967795: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

ModuleNotFoundError: No module named 'keras.src'

In [ ]:
# df = pd.read_csv('data/spotify_playlists.tsv', sep='\t', index_col=0)
# # df = df.sample(100)
# # assign a random similarity score
# df["similarity"] = np.random.rand(len(df))
df["duration_minutes"] = df['duration_ms'] / 60000

In [ ]:
df.columns

In [ ]:
bpm_range.value

In [ ]:
def apply_all_constraints(df):
    mode = {"Major": 1, "Minor" : 0}
    df = df[(df["tempo"] > bpm_range.value[0]) & (df["tempo"] < bpm_range.value[1])]
    df = df[(df["loudness"] > loudness_range.value[0]) & (df["loudness"] < loudness_range.value[1])]
    df = df[(df["liveness"] > liveness_range.value[0]) & (df["liveness"] < liveness_range.value[1])]
    df = df[df["mode"] == mode[mode_selection.value]]
    import pandas as pd

    def apply_constraints(dataframe, playlist_length, artist_name, constraint_option, k):
        selected_rows = pd.DataFrame(columns=dataframe.columns)
        current_sum = 0
        artist_count = 0

        if constraint_option == 'at least k':
            # Add top rows corresponding to the artist until reaching at least k
            artist_rows = dataframe[dataframe['artist_name'] == artist_name].head(k)
            selected_rows = pd.concat([selected_rows, artist_rows])
            current_sum = selected_rows['duration_minutes'].sum()
            artist_count = len(artist_rows)

        for index, row in dataframe.iterrows():
            if index not in selected_rows.index:
                
                if current_sum + row['duration_minutes'] <= playlist_length + 10:
                    # Check constraint 2: At most k, at least k, or none songs from the specified artist
                    if constraint_option == 'at most k' and artist_count < k:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']
                        if artist_name == row['artist_name']:
                            artist_count += 1
                    elif constraint_option == 'none' and artist_name != row['artist_name'] and artist_count == 0:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']
                    else:
                        selected_rows = selected_rows.append(row)
                        current_sum += row['duration_minutes']

                # Check constraint 1: Sum of column_1 within +/- 10 of the target_sum
                if current_sum >= playlist_length - 10 and current_sum <= playlist_length + 10:
                    break

        return selected_rows

    return apply_constraints(df, playlist_length=150, artist_name='Taylor Swift', constraint_option='at least k', k=3)
    # TODOOOOO: change the above arguments to the values from the togles ^

apply_all_constraints(df)

In [ ]:
# df["tempo"].describe()

In [70]:
# filtering_criterion = {
#     "Author": "SZA",
#     "ms_played": lambda x: x < 100_000
# }